### Assignment

In [4]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS,cross_origin
import logging
import os
import requests
import re
import pandas as pd




In [28]:
input_text = input("input youtube channel name")
url = f'https://www.youtube.com/@{input_text}/videos'
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0',
'Accept-Language': 'en-US,en;q=0.9'
}


response = requests.get(url, headers=headers)
response_text = response.text



vid_titles = re.findall('"title":{"runs":\[{"text":".*?"', response_text)


 YRF


In [29]:
vid_titles 

['"title":{"runs":[{"text":"Making of Jhoome Jo Pathaan Song | Pathaan | Shah Rukh Khan | Deepika Padukone | Siddharth Anand"',
 '"title":{"runs":[{"text":"Making of Besharam Rang Song | Pathaan | Shah Rukh Khan | Deepika Padukone | Siddharth Anand"',
 '"title":{"runs":[{"text":"World of Pathaan | Making Video | Shah Rukh Khan | Deepika Padukone | John Abraham | Siddharth Anand"',
 '"title":{"runs":[{"text":"Pathaan in Dubai | Making Video | Shah Rukh Khan | John Abraham | Siddharth Anand | In Cinemas Now"',
 '"title":{"runs":[{"text":"Pathaan Success Press Meet | Shah Rukh Khan | Deepika Padukone | John Abraham | Siddharth Anand"',
 '"title":{"runs":[{"text":"The Romantics | Official Trailer | Shah Rukh Khan, Salman Khan, Ranbir Kapoor | Netflix India"',
 '"title":{"runs":[{"text":"Pathaan Conversations with Sukriti Kakar | In Cinemas Now"',
 '"title":{"runs":[{"text":"Pathaan Conversations with Shilpa Rao | In Cinemas Now"',
 '"title":{"runs":[{"text":"Pathaan Conversations with Deep

In [30]:
vid_thumbnails = re.findall(r"https://i.ytimg.com/vi/[A-Za-z0-9_-]{11}/[A-Za-z0-9_]{9}.jpg", response_text)
vid_thumbnails

In [31]:
vid_links = re.findall(r"watch\?v=[A-Za-z0-9_-]{11}", response_text)
vid_links

In [32]:
pattern3 = re.compile(r"[0-9]+(\.[0-9]+)?[a-zA-Z]*K views")  # view count
pattern3

In [33]:
pattern4 = re.compile(r"\d+ (minutes|hours|hour|days|day|weeks|week|years|year) ago")  # vedio age
pattern4

In [34]:
matches1 = pattern3.finditer(response_text)
matches2 = pattern4.finditer(response_text)

In [35]:
vid_viewcounts=[]
vid_ages=[]
count = 0
for match1,match2 in zip(matches1,matches2):
    vid_ages.append(match2[0])
    vid_viewcounts.append(match1[0])

In [38]:
titles = vid_titles[0:10]
thumbnails = list(dict.fromkeys(vid_thumbnails))
links = vid_links[0:10]
viewcounts=vid_viewcounts[0:20:2]
ages=vid_ages[0:20:2]

details_list=[]

for title,thumbnail,link,viewcount,age in zip(titles,thumbnails,links,viewcounts,ages):
    details_dict={
    "title":title.split('"')[-2], "thumbnail": thumbnail, "link": "https://www.youtube.com/"+link,
    "viewcount": viewcount, "age": age
    }
    details_list.append(details_dict)



details_list

[{'title': 'Making of Jhoome Jo Pathaan Song | Pathaan | Shah Rukh Khan | Deepika Padukone | Siddharth Anand',
  'thumbnail': 'https://i.ytimg.com/vi/uSSFACVucbs/hqdefault.jpg',
  'link': 'https://www.youtube.com/watch?v=uSSFACVucbs',
  'viewcount': '379K views',
  'age': '3 weeks ago'},
 {'title': 'Making of Besharam Rang Song | Pathaan | Shah Rukh Khan | Deepika Padukone | Siddharth Anand',
  'thumbnail': 'https://i.ytimg.com/vi/p7f685ljJL8/hqdefault.jpg',
  'link': 'https://www.youtube.com/watch?v=p7f685ljJL8',
  'viewcount': '122K views',
  'age': '3 weeks ago'}]

In [37]:
df = pd.DataFrame(details_list)
df.to_csv('YTscrapData.csv', index=False)